In [1]:
'''
sevice elasticsearch start
'''

'\nsevice elasticsearch start\n'

In [226]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '45447f0361d8',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '4IrRB9I_QtSc9ZaomU5o5g',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [227]:
'''
curl -XGET localhost:9200/_cat/indices
'''

'\ncurl -XGET localhost:9200/_cat/indices\n'

# Wiki 파일 읽어오기 & 전처리

In [228]:
path = '/opt/ml/data/wikipedia_documents.json'
# path = '/opt/ml/data/wikipedia_documents_v3_no_remove.json'

In [229]:
import json

with open(path, 'r', encoding="utf-8") as f:
    wiki = json.load(f)

wiki['42']

{'text': '1971년 정주영 회장은 혼자서 미포만 해변 사진 한 장과 외국 조선소에서 빌린 유조선 설계도 하나 들고 유럽을 돌았다. 차관을 받기 위해서였다. 부정적인 반응만 받다가 1971년 9월 영국 바클레이 은행의 차관을 받기 위한 추천서를 부탁하기 위해 A&P 애플도어의 롱바톰 회장을 만났지만 대답은 역시 \'No\'였다. 이 때 정주영은 우리 나라 5백원짜리 지폐를 꺼내 거기 그려진 거북선 그림을 보여줬다. "우리는 영국보다 300년이나 앞선 1500년대에 이미 철갑선을 만들어 외국을 물리쳤소. 비록 쇄국정책으로 시기가 좀 늦어졌지만, 그 잠재력만큼은 충분하다고 생각하오."라며 설득해 결국 차관 도입에 성공할 수 있었다(이 부분은 너무 정주영씨를 미화한 것 같습니다. 사실로 믿고 싶을 만큼 멋진 내용입니다만 5백원짜리 지폐는 1973년 처음으로 발행되었기에 1971년 9월 차관과는 시계열상으로도 맞지 않습니다). \n\n1977년 서울 압구정동 현대아파트의 분양특혜사건으로 재판을 받았으나 무죄로 풀려났다. 건축법 위반에 대해 징역 6월 벌금 500만원에 선고유예 판결을 받았으나 현대산업개발 사장이었던 차남 정몽구가 서울지검 특수부에 구속되어 아들이 아버지 대신 처벌받는 전례가 만들어졌다.\n\n1978년에는 아산사회복지사업재단을 설립하였으며 같은 해 4월 29일 서울 강남구 압구정동에 위치한 현대고등학교를 설립하고 초대이사장으로 취임하였다. 1983년에는 현대전자주식회사를 설립하였다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '정주영',
 'author': None,
 'html': None,
 'document_id': 42}

# User_Dict_Rules 구성하기

In [230]:
titles = []
for doc in wiki:
    titles.append(wiki[doc]['title'])
titles = list(set(titles))

In [231]:
import re

title_list = []
for i, word in enumerate(titles):

    word_list = []
    # word_list.append(re.sub('\s', '', word))
    tmp = re.sub('[()]', '', word).split()
    for w in tmp:
        w = re.sub('~', '-', w)
        w = re.sub('/', '|', w)
        if len(w) == 1:
            continue
        elif re.search('[가-힣ㄱ-ㅎㅏ-ㅣ]', w) == None:
            continue
        elif re.search('년', w):
            continue
        else:
            title_list.append(w)

title_list = list(set(title_list))
if '' in title_list:
    title_list.pop(title_list.index(''))

In [232]:
title_list[:30]

['가수',
 '바우트웰',
 '퀸의',
 '기적비',
 '젯카이',
 '보경사',
 '포터와',
 '난계사',
 '기압계',
 '홍양명',
 '신전',
 '구리',
 '바톤',
 '카야니',
 '일필계상',
 '까삐깜',
 '손지환',
 '하피게이트',
 '구지라',
 '엔들리스',
 '와이카토',
 '빈맥',
 '한국연극배우협회',
 '의존증',
 '헤드램프',
 '장헌충',
 '에임스',
 '푸드',
 '금정사',
 '프린치프']

In [233]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type": "stop",
                        "stopwords_path": "stopkr.txt" # stopkr.txt 파일이 /etc/elasticsearch 에 있어야 함
                    }
                },
                "tokenizer": {
                    "my_nori_tokenizer": {
                        "type": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "user_dictionary_rules": title_list
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "my_nori_tokenizer", # 위에서 정의한 my_nori_tokenizer
                        "decompound_mode": "mixed",
                        "filter" : ["my_stop_filter"] # 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            # "dynamic": "strict",
            "properties": {
                # "title":{"type": "text", "analyzer": "nori_analyzer"},
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "document_id": {"type": "long"}
            }
        }
    }

In [234]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [235]:
import kss
from tqdm.auto import tqdm
from copy import deepcopy

def indexing_with_kss(data, es, add_title=True):

    wiki_ = deepcopy(data)

    for doc_id, doc in tqdm(wiki_.items(), total=len(wiki_)):
        for key in ['text', 'title']:
            doc[key] = re.sub('~', '-', doc[key]) # 쿼리에 ~가있어서 안댐
            doc[key] = re.sub('/', '|', doc[key])
        
        # doc['text'] = ''.join(['[', re.sub('\s', '', doc['title']), '] ', doc['text']])

        title_token = ''.join(['[', re.sub('\s', '', doc['title']), ']'])
        sents = kss.split_sentences(doc['text'])

        for i in range(len(sents)//2):
            even = True if len(sents)%2 == 0 else False
            last = len(sents)//2 - 1
            if add_title:
                if i == last and even:
                    sent = ' '.join([title_token, sents[2*i], sents[2*i+1]])
                else:
                    sent = ' '.join([title_token, sents[2*i], sents[2*i+1], sents[2*i+2]])
            else:
                if i == last and even:
                    sent = ' '.join([sents[2*i], sents[2*i+1]])
                else:
                    sent = ' '.join([sents[2*i], sents[2*i+1], sents[2*i+2]])

            es.index(index=INDEX_NAME, body=dict([('text', sent), ('title', doc['title'])]))

In [236]:
import re
from tqdm.auto import tqdm
from copy import deepcopy

def indexing_paragraph(data, es, add_title=True, min_length=100):
    
    def id_generator():
        id = 0
        while True:
            yield id
            id += 1
    
    wiki_ = deepcopy(data)
    id_gen = id_generator()

    for doc_id, doc in tqdm(wiki_.items(), total=len(wiki_)):
        for key in ['text', 'title']:
            doc[key] = re.sub('~', '-', doc[key]) # 쿼리에 ~가있어서 안댐
            doc[key] = re.sub('/', '|', doc[key])
        
        # doc['text'] = ''.join(['[', re.sub('\s', '', doc['title']), '] ', doc['text']])

        title_token = ''.join(['[',doc['title'], ']'])
        pars = re.split(r'\n\n', doc['text'])

        past_par = ''
        for paragraph in pars:
            paragraph = paragraph.strip()
            
            if len(' '.join([past_par, paragraph])) < min_length:
                past_par = ' '.join([past_par, paragraph])
                continue

            if add_title:
                paragraph = ' '.join([title_token, past_par, paragraph])
            else:
                paragraph = ' '.join([past_par, paragraph])

            past_par = ''
            es.index(index=INDEX_NAME, id=next(id_gen), body=dict([('text', paragraph), ('title', doc['title'])]))
        
        if past_par != '':
            if add_title:
                past_par = ' '.join([title_token, past_par])
            es.index(index=INDEX_NAME, id=next(id_gen), body=dict([('text', past_par), ('title', doc['title'])]))

In [238]:
import re
from tqdm.auto import tqdm
from copy import deepcopy

def indexing(data, es, add_title=True):
    
    wiki_ = deepcopy(data)

    for doc_id, doc in tqdm(wiki_.items(), total=len(wiki_)):
        for key in ['text', 'title']:
            doc[key] = re.sub('~', '-', doc[key]) # 쿼리에 ~가있어서 안댐
            doc[key] = re.sub('/', '|', doc[key])
        
        # doc['text'] = ''.join(['[', re.sub('\s', '', doc['title']), '] ', doc['text']])

        title_token = ''.join(['[',doc['title'], ']'])
        if add_title:
            doc['text'] = ' '.join([title_token, doc['text']])

        es.index(index=INDEX_NAME, id=doc_id, body=doc)

In [239]:
# indexing_with_kss(data=wiki, es=es, add_title=True)
# indexing_paragraph(data=wiki, es=es, add_title=False, min_length=128)
indexing(data=wiki, es=es, add_title=True)

  0%|          | 0/60613 [00:00<?, ?it/s]

In [240]:
# wiki_ = deepcopy(wiki)
# leng = []
# length = []
# for doc_id, doc in tqdm(wiki_.items(), total=len(wiki_)):
#     if re.search('\n\n', doc['text']):
#         leng.append(len(doc['text']))
#         continue
#     else:
#         length.append(len(doc['text']))

# print(sum(leng)/len(leng))
# print(sum(length)/len(length))

In [241]:
from pprint import pprint

doc = es.get(index=INDEX_NAME, id=42)
pprint(doc)

{'_id': '42',
 '_index': 'wiki',
 '_primary_term': 1,
 '_seq_no': 42,
 '_source': {'author': None,
             'corpus_source': '위키피디아',
             'document_id': 42,
             'domain': None,
             'html': None,
             'text': '[정주영] 1971년 정주영 회장은 혼자서 미포만 해변 사진 한 장과 외국 조선소에서 빌린 유조선 '
                     '설계도 하나 들고 유럽을 돌았다. 차관을 받기 위해서였다. 부정적인 반응만 받다가 1971년 9월 '
                     '영국 바클레이 은행의 차관을 받기 위한 추천서를 부탁하기 위해 A&P 애플도어의 롱바톰 회장을 '
                     "만났지만 대답은 역시 'No'였다. 이 때 정주영은 우리 나라 5백원짜리 지폐를 꺼내 거기 그려진 "
                     '거북선 그림을 보여줬다. "우리는 영국보다 300년이나 앞선 1500년대에 이미 철갑선을 만들어 '
                     '외국을 물리쳤소. 비록 쇄국정책으로 시기가 좀 늦어졌지만, 그 잠재력만큼은 충분하다고 생각하오."라며 '
                     '설득해 결국 차관 도입에 성공할 수 있었다(이 부분은 너무 정주영씨를 미화한 것 같습니다. 사실로 '
                     '믿고 싶을 만큼 멋진 내용입니다만 5백원짜리 지폐는 1973년 처음으로 발행되었기에 1971년 9월 '
                     '차관과는 시계열상으로도 맞지 않습니다). \n'
                     '\n'
                     '1977년 서울 압구정동 현대아파트의 분양특혜사건으로 재판을 받았으나 무죄로 풀려

In [242]:
query = "11-12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?"
# ~는 -으로, /는 없애버림
res = es.search(index=INDEX_NAME, q=query, size=10)
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '34146',
                    '_index': 'wiki',
                    '_score': 24.617607,
                    '_source': {'author': None,
                                'corpus_source': '위키피디아',
                                'document_id': 34146,
                                'domain': None,
                                'html': None,
                                'text': '[금동삼존불감] 불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 '
                                        '일반적인 건축물보다 작은 규모로 만든 것을 불감(佛龕)이라고 한다. '
                                        '불감은 그 안에 모신 불상의 양식뿐만 아니라, 당시의 건축 양식을 '
                                        '함께 살필 수 있는 중요한 자료가 된다. \\n\\n이 작품은 높이 '
                                        '18cm의 작은 불감으로, 청동으로 불감과 불상을 만들고 그 위에 '
                                        '금칠을 하였다. 불감 내부를 살펴보면 난간을 두른 사각형의 기단 '
                                        '위에 본존불과 양 옆에 보살상이 있으며, 그 위에 기둥과 지붕으로 '
  

In [243]:
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))# 문서를 참고해서 score

Doc ID: '34146'  Score: 24.62
Doc ID: '37924'  Score: 24.35
Doc ID: '37103'  Score: 19.94
Doc ID: '36554'  Score: 18.70
Doc ID: '45557'  Score: 18.47
Doc ID: '47125'  Score: 18.30
Doc ID: '37096'  Score: 18.20
Doc ID: '37967'  Score: 18.19
Doc ID: '40878'  Score: 17.88
Doc ID: '36816'  Score: 17.79


In [244]:
from datasets import load_from_disk

train_path = "/opt/ml/data/train_dataset"
train_data = load_from_disk(train_path)

train = train_data['train']
valid = train_data['validation']

In [245]:
for i in range(len(train)):
    print(train[i])
    break

{'title': '미국 상원', 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05', 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?', 'id': 'mrc-1-000067', 'answers': {'answer_start': [2

In [246]:
def es_retrieval(size=5):
    k = 0
    total_len = 0
    past_query = ''
    for i in tqdm(range(len(train))):
        query = train[i]['question']
        answers = train[i]['answers']['text']

        if query == past_query:
            continue
        else:
            total_len += 1
        past_query = query
        
        # answers = re.sub(r"[!/\\~\-\.\+“”%°〈〉<>＜＞・「」;*≪≫·《》‘’『』\"\'_]", '', answers)
        query = re.sub('~', '-', query) # 쿼리에 ~가있어서 안댐
        query = re.sub('/', '|', query)
        # print(query)
        res = es.search(index=INDEX_NAME, q=query, size=size)

        for hit in res['hits']['hits']:
            res_text = hit['_source']['text']
            found = False
            for answer in answers:
                if answer in res_text:
                    found = True
                    break
            if found == True:
                k += 1
                break


    return k, total_len

In [247]:
from elasticsearch import RequestError
from pprint import pprint

for size in [1, 5, 10, 15]:
    k, tot = 0, 0
    past_query = ''
    for i in tqdm(range(len(train))):
        query = train[i]['question']
        answers = train[i]['answers']['text']

        if query == past_query:
            continue
        else:
            tot += 1
        past_query = query
        
        # answers = re.sub(r"[!/\\~\-\.\+“”%°〈〉<>＜＞・「」;*≪≫·《》‘’『』\"\'_]", '', answers)
        query = re.sub('~', '-', query) # 쿼리에 ~가있어서 안댐
        query = re.sub('/', '|', query)
        # print(i, query, end='...  ')

        try:
            res = es.search(index=INDEX_NAME, q=query, size=size)
        except RequestError as e:
            # pprint(e.info)
            # break
            print('RequestError: ', i, query)
            continue

        # print('search done')
        
        for hit in res['hits']['hits']:
            res_text = hit['_source']['text']
            found = False
            for answer in answers:
                if answer in res_text:
                    found = True
                    break
            if found == True:
                k += 1
                break

    print(f'size {size}: probability {k/tot}, k = {k}, total = {tot}')

  0%|          | 0/3952 [00:00<?, ?it/s]

size 1: probability 0.7241902834008097, k = 2862, total = 3952


  0%|          | 0/3952 [00:00<?, ?it/s]

size 5: probability 0.8717105263157895, k = 3445, total = 3952


  0%|          | 0/3952 [00:00<?, ?it/s]

size 10: probability 0.9086538461538461, k = 3591, total = 3952


  0%|          | 0/3952 [00:00<?, ?it/s]

size 15: probability 0.9273785425101214, k = 3665, total = 3952
